<a href="https://colab.research.google.com/github/deniskapel/GenerativeChitChat/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir data
mkdir pretrained
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE" -O data/qa_data.jsonl.zip && rm -rf /tmp/cookies.txt
unzip data/qa_data.jsonl.zip -d data

In [ ]:
%%bash
wget "https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt"
pip install -r requirements.txt

In [244]:
import codecs
import json
import random
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.nn.utils.rnn import pad_sequence

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [6]:
assert torch.cuda.is_available(), 'no gpu available'
device = torch.device('cuda')
device

device(type='cuda')

In [ ]:
tokenizer = yttm.BPE(model="pretrained/my_pretrained_bpe_lm.model")

In [483]:
batch_size = 64
pad_index = 0
eos_index = 3
vocab_size=30_000

## Data

In [7]:
!sed 5q data/qa_data.jsonl

{"question": "долго ли идут деньги с яндексденег на карту visa?", "category": "Бизнес, Финансы", "responses": ["нет. прорыв 35 ;)"]}
{"question": "можно ли зарегистрировать авто в другом регионе", "category": "Авто, Мото", "responses": ["можно на родственника из того региона.. .  а потом ездить по доверке"]}
{"question": "что делать если у меня очень тонкие ногти а хочется их отрастить?", "category": "Красота и Здоровье", "responses": ["витамины и умная эмаль (каждый день)", "ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.", "умная эмаль, витамины, йод, и поменьше крась лаком ", "лаки фирмы trind производство usa + кальций"]}
{"question": "в чем отличие медитации от йоги?", "category": "Спорт", "responses": ["букв в йоге меньше", "в медитации ты просто сидишь и мммммычишь. а в йоге всяко разные упражнения вытворяешь", "в медитации вроде просто тупо сидишь и успокаеваешься, а в йоге еще и ноги за уши закидывать надо"]}
{"question": "когда нач

In [8]:
with codecs.open("data/qa_data.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))

data = []
for line in tqdm(lines):
    for response in line['responses']:
        data.append(
            {'question': line['question'].lower(),
             'category': line['category'],
             'response': response.lower()})

del lines
df = pd.json_normalize(data)
del data

  0%|          | 0/2808811 [00:00<?, ?it/s]

In [9]:
df.head()

,question,category,response
0,долго ли идут деньги с яндексденег на карту visa?,"Бизнес, Финансы",нет. прорыв 35 ;)
1,можно ли зарегистрировать авто в другом регионе,"Авто, Мото",можно на родственника из того региона.. . а п...
2,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,витамины и умная эмаль (каждый день)
3,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,ванночки с морской солью. с вечера мажь ногти ...
4,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,"умная эмаль, витамины, йод, и поменьше крась л..."


In [38]:
df_mini = df.sample(frac=0.01, random_state=42)
train_df = df_mini.sample(frac=0.8,random_state=42)
val_df = df_mini.drop(train_df.index)
test_df = val_df.sample(frac=0.5,random_state=42)
val_df = val_df.drop(test_df.index)

In [ ]:
val_df.category.value_counts()

### Datasets

In [578]:
class QAData(torch.utils.data.Dataset):
    
    def __init__(self, data: pd.DataFrame, tokenizer,
                 pad_index=0, eos_index=3, response_len=32):
        
        self.x = data.question.tolist()
        self.y = data.response.tolist()
        self.tokenizer = tokenizer
        # to use with beam search later
        self.categories = data.category.tolist()
        self.response_maxlen = response_len
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        """ x is a question and y is an asnwer """
        x = self.tokenizer.encode(self.x[index], bos=True, eos=True)
        y = self.tokenizer.encode(self.y[index], bos=True, eos=True)
        # as vocab_size=30_000, use int16 as its largest value is 32_767
        x = torch.Tensor(x).type(torch.int)
        
        y = y[:self.response_maxlen]
        pads = [self.pad_index] * (self.response_maxlen - len(y))
        y = y + pads
        
        return x, y 

    def collate_batch(self, batch):
        """
        add padding to dynamically match the longest sentence in a batch
        """
        batch_x = []
        batch_y = []
        for sample in batch:
            batch_x.append(sample[0])
            batch_y.append(sample[1])

        batch_x = pad_sequence(batch_x, padding_value=self.pad_index, batch_first=True)
        batch_y = torch.Tensor(batch_y).type(torch.int)

        return batch_x, batch_y

In [579]:
train_dataset = QAData(
    data=train_df,
    tokenizer=tokenizer)

val_dataset = QAData(
    data=val_df,
    tokenizer=tokenizer)

test_dataset = QAData(
    data=test_df,
    tokenizer=tokenizer)

len(train_dataset), len(val_dataset), len(test_dataset)

(62137, 7767, 7767)

### DataLoader

Migrate torchtext from the legacy API to the new API
[tutorial](https://colab.research.google.com/github/pytorch/text/blob/master/examples/legacy_tutorial/migration_tutorial.ipynb#scrollTo=Jky4X-iFU4HK)

In [580]:
class Sampler():
    def __init__(self, dataset, batch_size=64):
        self.dataset = dataset
        self.n_batches = len(dataset) // batch_size
        self.batch_size = batch_size

    def __iter__(self):
        indices = [(i, len(s[0])) for i, s in enumerate(self.dataset)]
        random.shuffle(indices)
        pooled_indices = []
        # create pool of indices with similar lengths 
        for i in range(0, len(indices), batch_size * 100):
            pooled_indices.extend(
                sorted(indices[i:i + batch_size * 100], key=lambda x: x[1])
                )
        pooled_indices = [x[0] for x in pooled_indices]
        
        # yield indices for current batch
        for i in range(0, len(pooled_indices), batch_size):
            yield pooled_indices[i:i + batch_size]

train_sampler = Sampler(train_dataset)
val_sampler = Sampler(val_dataset)
test_sampler = Sampler(test_dataset)

In [581]:
train_loader = DataLoader(
    train_dataset,
    collate_fn=train_dataset.collate_batch,
    batch_sampler=train_sampler)

val_loader = DataLoader(
    val_dataset,
    collate_fn=val_dataset.collate_batch,
    batch_sampler=val_sampler)

test_loader = DataLoader(
    test_dataset,
    collate_fn=val_dataset.collate_batch,
    batch_sampler=test_sampler)

In [582]:
for batch in train_loader:
    break

batch[0].shape

torch.Size([64, 5])

In [583]:
train_sampler.n_batches,val_sampler.n_batches,test_sampler.n_batches

(970, 121, 121)

In [584]:
progress_bar = tqdm(total=len(train_loader.dataset), desc='Testing')

for x, y in train_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/62137 [00:00<?, ?it/s]

In [585]:
progress_bar = tqdm(total=len(val_loader.dataset), desc='Testing')

for x, y in val_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/7767 [00:00<?, ?it/s]

In [586]:
progress_bar = tqdm(total=len(test_loader.dataset), desc='Testing')

for x, y in test_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/7767 [00:00<?, ?it/s]